In [ ]:
import numpy as np

from section4_1 import \
    get_Q_T_H_A_d_t, \
    get_Q_UT_H_A_d_t, \
    get_Q_T_CS_A_d_t, \
    get_Q_T_CL_A_d_t

from section11_1 import \
    load_outdoor, \
    get_T_ex, \
    get_Theta_ex, \
    get_X_ex, \
    get_h_ex

## 第二節　ダクト式セントラル空調機

1．適用範囲～4．記号及び単位は省略

### 5．最大暖房出力

$$\large
Q_{max,H,d,t} = q_{rtd,H} \times C_{df,H,d,t} \times 3600 \times 10^{-6}
$$

$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h)  
$q_{rtd,H}$：定格暖房能力(W)   
$C_{df,H,d,t}$：日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数  

日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数$C_{df,H,d,t}$は、外気温度が5℃未満かつ相対湿度が80%以上の場合にデフロストが入ると仮定し、その場合の値を0.77とし、それ以外の条件においては1.0とする。

In [ ]:
# 最大暖房出力 (1)
def get_Q_max_H_d_t(region, q_rtd_H):
    # 外気条件
    outdoor = load_outdoor()
    Theta_ex = get_Theta_ex(region, outdoor)
    T_ex = get_T_ex(Theta_ex)
    X_ex = get_X_ex(region, outdoor)
    h_ex = get_h_ex(X_ex, Theta_ex)

    # デフロストに関する暖房出力補正係数
    C_df_H_d_t = np.ones(24 * 365)
    C_df_H_d_t[(Theta_ex < 5.0) * (h_ex > 80.0)] = 0.77

    return q_rtd_H * C_df_H_d_t * 3600 / 1000000

### 6．暖房エネルギー消費量

#### 6.1　消費電力量

$Qr'_{T,H,d,t} \geq 0.3$ の場合

$$
\large
E_{E,H,d,t} = ( a_1 \times ( b_1 \times \theta_{ex,d,t} + b_2 \times ln(Qr'_{T,H,d,t}) + b_3 )^{a2}  \\
\large
+ c_1 \times \Biggl( \frac{Qr'_{T,H,d,t}}{d_1 \times \theta_{ex,d,t} + d_2 \times ln(Qr'_{T,H,d,t}) + d_3} \Biggr)^{c_2} \Biggr)
\times P_{rtd,H} \times 10^{-3} \times C_{pl,H}
$$

$Qr'_{T,H,d,t} < 0.3$ の場合

$$
\large
E_{E,H,d,t} = ( a_1 \times ( b_1 \times \theta_{ex,d,t} + b_2 \times ln(0.3) + b_3 )^{a_2}  \\
\large
+ c_1 \times \Biggl( \frac{0.3}{d_1 \times \theta_{ex,d,t} + d_2 \times ln(0.3) + d_3} \Biggr)^{c_2} \Biggr)
\times P_{rtd,H} \times \frac{Qr'_{T,H,d,t}}{0.3} \times 10^{-3} \times C_{pl,H}
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  
$Qr'_{T,H,d,t}$：日付$d$の時刻$t$における補正処理暖房負荷比    
$P_{rtd,H}$：定格暖房消費電力(W)  
$C_{pl,H}$：暖房時におけるダクト等圧力損失による消費電力量補正係数  

<div style="text-align: center;font-weight: bold;">表　係数</div>

|係数|値|係数|値|
|-|-|-|-|
|$a_1$|0.0003124|$c_1$|0.003955|
|$a_2$|1.923|$c_2$|2.001|
|$b_1$|-0.7945|$d_1$| 7.442 $\times 10^{-5}$|
|$b_2$|21.54|$d_2$|0.09156|
|$b_3$|73.02|$d_3$|0.2344|


In [ ]:
def get_E_E_H_d_t(region, q_rtd_H, P_rtd_H, C_pl_H, L_H_d_t):
    outdoor = load_outdoor()

    # 表3 係数
    a1 = 0.0003124
    a2 = 1.923
    b1 = -0.7945
    b2 = 21.54
    b3 = 73.02
    c1 = 0.003955
    c2 = 2.001
    d1 = 7.442 * 10 ** (-5)
    d2 = 0.09156
    d3 = 0.2344

    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(region, q_rtd_H)

    # 処理負荷
    Q_T_H_d_t = get_Q_T_H_A_d_t(Q_max_H_d_t, L_H_d_t)
    print('L_T_H: {} [MJ/年]\nQ_T_H: {} [MJ/年]'.format(np.sum(L_H_d_t), np.sum(Q_T_H_d_t)))

    # 補正処理暖房負荷比
    Q_r_dash_T_H_d_t = get_Q_r_dash_T_H_d_t(q_rtd_H, region, outdoor, Q_T_H_d_t)

    # 外気条件
    Theta_ex_d_t = get_Theta_ex(region, outdoor)

    tmp_2a = np.zeros(24*365)
    f2a = Q_r_dash_T_H_d_t >= 0.3
    tmp_2a[f2a] = (a1 * np.power(b1 * Theta_ex_d_t[f2a] + b2 * np.log(Q_r_dash_T_H_d_t[f2a]) + b3, a2)
                   + c1 * np.power(Q_r_dash_T_H_d_t[f2a] / (d1 * Theta_ex_d_t[f2a] + d2 * np.log(Q_r_dash_T_H_d_t[f2a]) + d3),
                              c2)) * P_rtd_H * 10 ** (-3) * C_pl_H

    tmp_2b = np.zeros(24*365)
    f2b = Q_r_dash_T_H_d_t < 0.3
    tmp_2b[f2b] = (a1 * np.power(b1 * Theta_ex_d_t[f2b] + b2 * np.log(0.3) + b3, a2)
                   + c1 * np.power(0.3 / (d1 * Theta_ex_d_t[f2b] + d2 * np.log(0.3) + d3), c2)) * P_rtd_H * (
                         Q_r_dash_T_H_d_t[f2b] / 0.3) * 10 ** (-3) * C_pl_H

    E_E_H_d_t = tmp_2a + tmp_2b

    return E_E_H_d_t

$$\large
Qr'_{T,H,d,t} = \frac{Q'_{T,H,d,t}}{q_{rtd,H} \times 3600 \times 10^{-6}}
$$

$q_{rtd,H,d,t}$：定格暖房能力(W)  
$Q'_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの補正処理暖房負荷(MJ/h)    

In [ ]:
# 補正処理暖房負荷比 (3)
def get_Q_r_dash_T_H_d_t(q_rtd_H, region, outdoor, Q_T_H_d_t):
    # 補正処理暖房負荷
    Q_dash_T_H_d_t = get_Q_dash_T_H_d_t(region, outdoor, Q_T_H_d_t)

    return Q_dash_T_H_d_t / (q_rtd_H * 3600 / 1000000)

$$\large
Q'_{T,H,d,t} = \frac{Q_{T,H,d,t}}{C_{df,H,d,t}}
$$

$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの処理暖房負荷(MJ/h)  
$C_{df,H,d,t}$：日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数    

In [ ]:
# 補正処理暖房負荷 (4)
def get_Q_dash_T_H_d_t(region, outdoor, Q_T_H_d_t):
    # デフロストに関する暖房出力補正係数
    C_df_H_d_t = get_C_df_H_d_t(region, outdoor)

    # 補正処理暖房負荷
    Q_dash_T_H_d_t = Q_T_H_d_t / C_df_H_d_t

    return Q_dash_T_H_d_t

日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数$C_{df,H,d,t}$は、外気温度が5℃未満かつ相対湿度が80%以上の場合にデフロストが入ると仮定し、その場合の値を0.77とし、それ以外の条件においては1.0とする。

In [ ]:
# デフロストに関する暖房出力補正係数
def get_C_df_H_d_t(region, outdoor):
    # 外気条件
    Theta_ex = get_Theta_ex(region, outdoor)
    X_ex = get_X_ex(region, outdoor)
    h_ex = get_h_ex(X_ex, Theta_ex)

    # 外気温度が5℃未満かつ相対湿度が80%以上の場合にデフロストが入ると仮定
    C_df_H_d_t = np.ones(24 * 365)
    C_df_H_d_t[(Theta_ex < 5.0) * (h_ex > 80.0)] = 0.77

    return C_df_H_d_t

#### 6.2　ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,H,d,t}$は0とする。

In [ ]:
def get_E_G_H_d_t():
    # ガス消費量は0とする
    return np.zeros(24 * 365)

#### 6.3　灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,H,d,t}$は0とする。

In [ ]:
def get_E_K_H_d_t():
    # 灯油消費量は0とする
    return np.zeros(24 * 365)

#### 6.4　その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,H,d,t}$は0とする。

In [ ]:
def get_E_M_H_d_t():
    # その他の燃料による一次エネルギー消費量は0とする
    return np.zeros(24 * 365)

### 7．最大冷房出力

#### 7.1　最大冷房出力の計算

$$\large
Q_{max,C,d,t} = q_{rtd,C} \times 3600 \times 10^{-6}
$$

$Q_{max,C,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房出力(MJ/h)  
$q_{rtd,C}$：定格冷房能力(W)  

In [ ]:
# 最大冷房出力 (5)
def get_Q_max_C_d_t(q_rtd_C):
    return np.ones(24 * 365) * (q_rtd_C * 3600 / 1000000

#### 7.2　最大冷房顕熱出力及び最大冷房潜熱出力の計算

$$\large
Q_{max,CS,d,t} = Q_{max,C,d,t} \times SHF'_{d,t}
$$

$$\large
Q_{max,CL,d,t} = min(Q_{max,C,d,t} \times (1 - SHF'_{d,t}),  L'_{CL,d,t})
$$

$Q_{max,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房顕熱出力(MJ/h)  
$Q_{max,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房潜熱出力(MJ/h)  
$SHF'_{d,t}$：日付$d$の時刻$t$における冷房負荷補正顕熱比  
$L'_{CL,d,t}$：日付$d$の時刻$t$における1時間当たりの補正冷房潜熱負荷(MJ/h)  
$Q_{max,C,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房出力(MJ/h)  

In [ ]:
# 1時間当たりの最大冷房顕熱出力
def get_Q_max_CS_d_t(Q_max_C_d_t, SHF_dash_d_t):
    return Q_max_C_d_t * SHF_dash_d_t


# 1時間当たりの最大冷房潜熱出力
def get_Q_max_CL_d_t(Q_max_C_d_t, SHF_dash_d_t, L_dash_CL_d_t):
    return np.min([Q_max_C_d_t * (1.0 - SHF_dash_d_t), L_dash_CL_d_t], axis=0)

$$\large
SHF'_{d,t} = \frac{L_{CS,d,t}}{L'_{C,d,t}}
$$

$L_{CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷(MJ/h)  
$L'_{C,d,t}$：日付$d$の時刻$t$における1時間当たりの補正冷房負荷(MJ/h)  

日付$d$の時刻$t$における１時間当たりの補正冷房負荷$L'_{C,d,t}$が0の場合、$SHF'_{d,t}$ = 0とする。

In [ ]:
# 冷房負荷補正顕熱比 (7)
def get_SHF_dash_d_t(L_CS_d_t, L_dash_C_d_t):
    SHF_dash_d_t = np.zeros(24*365)

    f = L_dash_C_d_t != 0
    SHF_dash_d_t[f] = L_CS_d_t[f] / L_dash_C_d_t[f]

    return SHF_dash_d_t

$$\large
L'_{c,d,t} = L_{CS,d,t} + L'_{CL,d,t}
$$

$$\large
L'_{CL,d,t} = min(L_{max,CL,d,t},  L_{CL,d,t})
$$

$$\large
L_{max,CL,d,t} = L_{CS,d,t} \times \frac{1 - SHF_{L,min,C}}{SHF_{L,min,C}}
$$

$L_{max,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房潜熱負荷(MJ/h)  
$L_{CL,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房潜熱負荷(MJ/h)  
$SHF_{L,min,C}$：冷房負荷最小顕熱比  

In [ ]:
# 1時間当たりの補正冷房負荷 (8)
def get_L_dash_C_d_t(L_CS_d_t, L_dash_CL_d_t):
    return L_CS_d_t + L_dash_CL_d_t


# 1時間当たりの補正冷房潜熱負荷 (9)
def get_L_dash_CL_d_t(L_max_CL_d_t, L_CL_d_t):
    return np.min([L_max_CL_d_t, L_CL_d_t], axis=0)


# 1時間当たりの最大冷房潜熱負荷 (10)
def get_L_max_CL_d_t(L_CS_d_t):
    # 冷房負荷最小顕熱比率
    SHF_L_min_C = get_SHF_L_min_C()

    return L_CS_d_t * (1.0 - SHF_L_min_C) / SHF_L_min_C

冷房負荷最小顕熱比$SHF_{L,min,C}$は0.4 とする。

In [ ]:
# 冷房負荷最小顕熱比率
def get_SHF_L_min_C():
    return 0.4

### 8．冷房エネルギー消費量

#### 8.1　消費電力量

$Qr_{T,C,d,t} \geq 0.3$ の場合

$$\large
E_{E,C,d,t} = (a_1 \times (b)^{a_2} + c_1 \times (d)^{c_2}) \times P_{rtd,C} \times 10^{-3} \times C_{pl,C}
$$

$$\large
\left\{ \begin{array}{ll}
    b = max(\theta_{ex,d,t} - b_1 \times \theta_{ex,d,t} - b_2 \times ln(X_{ex,d,t}) - b_3 \times ln(Qr_{T,C,d,t}) - b_{4},0)\\ 
    d = exp(d_1 \times \theta_{ex,d,t} + d_2 \times ln(X_{ex,d,t}) + d_3 \times ln(Qr_{T,C,d,t}) + d_4)\\ 
    \end{array}
\right.
$$

$Qr_{T,C,d,t} < 0.3$ の場合

$$\large
E_{E,C,d,t} = (a_1 \times (b)^{a_2} + c_1 \times (d)^{c_2}) \times P_{rtd,C} \times \frac{Qr_{T,C,d,t}}{0.3} \times 10^{-3} \times C_{pl,C}
$$

$$\large
\left\{ \begin{array}{ll}
    b = max(\theta_{ex,d,t} - b_1 \times \theta_{ex,d,t} - b_2 \times ln(X_{ex,d,t}) - b_3 \times ln(0.3) - b_{4},0)\\ 
    d = exp(d_1 \times \theta_{ex,d,t} + d_2 \times ln(X_{ex,d,t}) + d_3 \times ln(0.3) + d_4)\\ 
    \end{array}
\right.
$$

$E_{E,C,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)    
$X_{ex,d,t}$：日付$d$の時刻$t$における外気絶対湿度(kg/kg(DA))    
$Qr_{T,C,d,t}$：日付$d$の時刻$t$における処理冷房負荷比    
$P_{rtd,C}$：定格冷房消費電力(W)    
$C_{pl,C}$：冷房時におけるダクト等圧力損失による消費電力量補正係数  

<div style="text-align: center;font-weight: bold;">表　係数</div>

|係数|値|係数|値|
|-|-|-|-|
|$a_1$|0.001052|$c_1$|0.003939|
|$a_2$|2.051|$c_2$|1.058|
|$b_1$|0.241|$d_1$|0.0593|
|$b_2$|0.04985|$d_2$|0.08193|
|$b_3$|0.2083|$d_3$|1.257|
|$b_4$|−2.391|$d_4$|1.586|


In [ ]:
def get_E_E_C_d_t(region, q_rtd_C, P_rtd_C, C_pl_C, L_CS_d_t, L_CL_d_t):
    # 表4 式(11))中の係数
    a1 = 0.001052
    a2 = 2.051
    b1 = 0.241
    b2 = 0.04985
    b3 = 0.2083
    b4 = -2.391
    c1 = 0.003939
    c2 = 1.058
    d1 = 0.0593
    d2 = 0.08193
    d3 = 1.257
    d4 = 1.586

    # 外気条件
    outdoor = load_outdoor()
    Theta_ex_d_t = get_Theta_ex(region, outdoor)
    X_ex_d_t = get_X_ex(region, outdoor) * 10 ** (-3)

    # 最大冷房出力
    Q_max_C_d_t = get_Q_max_C_d_t(q_rtd_C)

    # 最大冷房潜熱負荷
    L_max_CL_d_t = get_L_max_CL_d_t(L_CS_d_t)

    # 補正冷房潜熱負荷
    L_dash_CL_d_t = get_L_dash_CL_d_t(L_max_CL_d_t, L_CL_d_t)
    L_dash_C_d_t = get_L_dash_C_d_t(L_CS_d_t, L_dash_CL_d_t)

    # 冷房負荷補正顕熱比
    SHF_dash_d_t = get_SHF_dash_d_t(L_CS_d_t, L_dash_C_d_t)

    # 最大冷房顕熱出力, 最大冷房潜熱出力
    Q_max_CS_d_t = get_Q_max_CS_d_t(Q_max_C_d_t, SHF_dash_d_t)
    Q_max_CL_d_t = get_Q_max_CL_d_t(Q_max_C_d_t, SHF_dash_d_t, L_dash_CL_d_t)

    # 処理顕熱負荷、処理潜熱負荷
    Q_T_CS_d_t = get_Q_T_CS_A_d_t(Q_max_CS_d_t, L_CS_d_t)
    Q_T_CL_d_t = get_Q_T_CL_A_d_t(Q_max_CL_d_t, L_CL_d_t)

    # 処理冷房負荷
    Q_T_C_d_t = get_Q_T_C_d_t(Q_T_CS_d_t, Q_T_CL_d_t)

    # 処理冷房負荷比
    Q_r_T_C_d_t = get_Q_r_T_C_d_t(q_rtd_C, Q_T_C_d_t)

    # (11a)
    E_E_C_d_t = np.zeros(24*365)
    f_11a = Q_r_T_C_d_t >= 0.3
    b = np.clip(Theta_ex_d_t[f_11a] - b1 * Theta_ex_d_t[f_11a] - b2 * np.log(X_ex_d_t[f_11a]) - b3 * np.log(Q_r_T_C_d_t[f_11a]) - b4, 0, None)
    d = np.exp(d1 * Theta_ex_d_t[f_11a] + d2 * np.log(X_ex_d_t[f_11a]) + d3 * np.log(Q_r_T_C_d_t[f_11a]) + d4)
    E_E_C_d_t[f_11a] = (a1 * np.power(b, a2) + c1 * np.power(d, c2)) * P_rtd_C * 10 ** (-3) * C_pl_C

    # (11b)
    f_11b = np.logical_and(0 < Q_r_T_C_d_t, Q_r_T_C_d_t < 0.3)
    b = np.clip(Theta_ex_d_t[f_11b] - b1 * Theta_ex_d_t[f_11b] - b2 * np.log(X_ex_d_t[f_11b]) - b3 * np.log(0.3) - b4, 0, None)
    d = np.exp(d1 * Theta_ex_d_t[f_11b] + d2 * np.log(X_ex_d_t[f_11b]) + d3 * np.log(0.3) + d4)
    E_E_C_d_t[f_11b] = (a1 * np.power(b, a2) + c1 * np.power(d, c2)) * P_rtd_C * (Q_r_T_C_d_t[f_11b] / 0.3) * 10 ** (-3) * C_pl_C

    return E_E_C_d_t

$$\large
Qr_{T,C,d,t} = \frac{Q_{T,C,d,t}}{q_{rtd,C} \times 3600 \times 10^{-6}}
$$

$Q_{T,C,d,t}$：日付$d$の時刻$t$における1時間当たりの処理冷房負荷(MJ/h)  
$q_{rtd,C}$：定格冷房能力(W)

In [ ]:
# 処理冷房負荷比 (12)
def get_Q_r_T_C_d_t(q_rtd_C, Q_T_C_d_t):
    return Q_T_C_d_t / (q_rtd_C * 3600 / 1000000)

$$\large
Q_{T,C,d,t} = Q_{T,CS,d,t} + Q_{T,CL,d,t}
$$

$Q_{T,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの処理冷房顕熱負荷(MJ/h)  
$Q_{T,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの処理冷房潜熱負荷(MJ/h)  

In [ ]:
# 処理冷房負荷 (13)
def get_Q_T_C_d_t(Q_T_CS_d_t, Q_T_CL_d_t):
    return Q_T_CS_d_t + Q_T_CL_d_t

#### 8.2　ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,C,d,t}$は0とする。

In [ ]:
def get_E_G_C_d_t():
    # ガス消費量は0とする
    return np.zeros(24 * 365)

#### 8.3　灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,C,d,t}$は0とする。

In [ ]:
def get_E_K_C_d_t():
    # 灯油消費量は0とする
    return np.zeros(24 * 365)

#### 8.4　その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,C,d,t}$は0とする。

In [ ]:
def get_E_M_C_d_t():
    # その他の燃料による一次エネルギー消費量は0とする
    return np.zeros(24 * 365)

## 付録 A 機器の性能を表す仕様の決定方法

### A.1　定格能力

$$\large
q_{rtd,H} = q_{rq,H} \times A_{A} \times f_{CT} \times f_{CL}
$$

$$\large
q_{rtd,C} = q_{rq,C} \times A_{A} \times f_{CT} \times f_{CL}
$$

$q_{rtd,H}$：定格暖房能力(W)  
$q_{rtd,C}$：定格冷房能力(W)  
$q_{rq,H}$：単位面積当たりの必要暖房能力(W/m<sup>2</sup>)  
$q_{rq,C}$：単位面積当たりの必要冷房能力(W/m<sup>2</sup>)  
$A_{A}$：床面積の合計(m<sup>2</sup>)   
$f_{CT}$：外気温度能力補正係数   
$f_{CL}$：間歇運転能力補正係数  

In [ ]:
# 定格暖房能力(W)
def get_q_rtd_H(region, A_A):
    q_rq_H = get_q_rq_H(region)
    return q_rq_H * A_A * f_CT * f_CL 


# 定格冷房能力(W)
def get_q_rtd_C(region, A_A):
    q_rq_C = get_q_rq_C(region)
    return q_rq_C * A_A * f_CT * f_CL

# 単位面積当たりの必要暖房能力
def get_q_rq_H(region):
    return table_a_1[0][region - 1]


# 単位面積当たりの必要冷房能力
def get_q_rq_C(region):
    return table_a_1[1][region - 1]

外気温度能力補正係数$f_{CT}$は、1.05の値とする。間歇運転能力補正係数$f_{CL}$は、1.0とする。

In [ ]:
# 外気温度能力補正係数
f_CT = 1.05

# 間歇運転能力補正係数
f_CL = 1.0

<div style="text-align: center;font-weight: bold;">表　単位面積当たりの必要暖房能力$q_{rq,H}$及び冷房能力$q_{rq,C}$(W/m<sup>2</sup>)</div>

|変数＼地域の区分|1|2|3|4|5|6|7|8|
|-|-|-|-|-|-|-|-|-|
|$q_{rq,H}$|73.91|64.32|62.65|66.99|72.64|61.34|64.55|－|
|$q_{rq,C}$|37.61|36.55|42.34|54.08|61.69|60.79|72.53|61.56|


In [ ]:
# 表A.1 単位面積当たりの必要暖房能力及び冷房能力（W/m2)
table_a_1 = [
    (73.91, 64.32, 62.65, 66.99, 72.64, 61.34, 64.55, 00.00),
    (37.61, 36.55, 42.34, 54.08, 61.69, 60.79, 72.53, 61.56)
]

### A.2　定格消費電力

$$\large
P_{rtd,H} = \frac{q_{rtd,H}}{e_{rtd,H}}
$$

$$\large
P_{rtd,C} = \frac{q_{rtd,C}}{e_{rtd,C}}
$$

$P_{rtd,H}$：定格暖房消費電力(W)  
$P_{rtd,C}$：定格冷房消費電力(W)  
$q_{rtd,H}$：定格暖房能力(W)  
$q_{rtd,C}$：定格冷房能力（(W)  
$e_{rtd,H}$：定格暖房エネルギー消費効率  
$e_{rtd,C}$：：定格冷房エネルギー消費効率  

In [ ]:
# 定格暖房消費電力
def get_P_rtd_H(q_rtd_H):
    return q_rtd_H / e_rtd_H  # (2a)


# 定格冷房消費電力
def get_P_rtd_C(q_rtd_C):
    return q_rtd_C / e_rtd_C  # (2b)

定格暖房エネルギー消費効率$e_{rtd,H}$は3.76、定格冷房エネルギー消費効率$e_{rtd,C}$は3.17とする。

In [ ]:
# 定格暖房エネルギー消費効率
e_rtd_H = 3.76

# 定格冷房エネルギー消費効率
e_rtd_C = 3.17

### A.3　複数のダクト式セントラル空調機が設置される場合の仕様の決定方法

省略

## 付録 B ダクト等圧力損失及び断熱区画外ダクト熱損失による消費電力量補正係数

暖房時におけるダクト等圧力損失による消費電力量補正係数$C_{pl,H}$及び冷房時におけるダクト等圧力損失による消費電力量補正係数$C_{pl,C}$は、圧力損失によって送風機の出力が変わらないタイプ（以下、「風量補正なし」という。）の場合、B.1 に示される暖房時におけるダクト等圧力損失による消費電力量補正係数（風量補正なし）$C_{pl,nm,H}$及び冷房時におけるダクト等圧力損失による消費電力量補正係数（風量補正なし）$C_{pl,nm,C}$に等しいとし、圧力損失によって送風機の出力が変わるタイプ（以下、「風量補正あり」という。）の場合、B.2 に示される暖房時におけるダクト等圧力損失による消費電力量補正係数（風量補正あり）$C_{pl,va,H}$及び冷房時におけるダクト等圧力損失による消費電力量補正係数（風量補正あり）$C_{pl,va,C}$に等しいとする。  
風量補正の有無が不明な場合は、「風量補正なし」とみなす。  
ダクトの全部または一部が断熱区画外に設置される場合は、B.1 又は B.2 により求めた係数にさらに B.3 に示す補正係数を乗じなければならない。

### B.1 風量補正なしの場合

暖房時におけるダクト等圧力損失による消費電力量補正係数（風量補正なし）$C_{pl,nm,H}$に 1.22、冷房時におけるダクト等圧力損失による消費電力量補正係数（風量補正なし）$C_{pl,nm,C}$に 1.25 を用いるか、ダクト等の圧力損失計算により算定される設計風量と定格風量との比により決定される。

### B.2 風量補正ありの場合

暖房時におけるダクト等圧力損失による消費電力量補正係数（風量補正あり）$C_{pl,va,H}$に1.03、冷房時におけるダクト等圧力損失による消費電力量補正係数（風量補正あり）$C_{pl,va,C}$に 1.03 を用いるか、ダクト等の圧力損失計算により算定される設計圧力損失と標準機外静圧との比により決定される。

### B.3 ダクトの全部または一部が断熱区画外に設置される場合の補正係数

表に示す条件に適合する場合は1.10、それ以外の場合は1.35とする。

<div style="text-align: center;font-weight: bold;">表　補正係数に1.10を適用できる条件</div>

|断熱区画外のダクトの設置位置|補正係数に 1.10 を適用できる条件|
|-|-|
|全部または一部が小屋裏<br>（小屋裏が断熱区画外の場合）<br>に設置されている場合|断熱区画外のダクトの長さ17.3m 以下<br>かつ断熱区画外の全ての部分において<br>線熱損失係数が0.49W/(mK)以下であること。|
|全部が床下<br>（床下が断熱区画外の場合）<br>に設置されている場合|断熱区画外のダクトの長さ23.3m 以下<br>かつ断熱区画外の全ての部分において<br>線熱損失係数が0.49W/(mK)以下であること|
